In [0]:
# %pip install semantic-link-labs

**PROMO SEMANTIC MODEL REFRESH THROUGH PARTITIONS POC**

In [0]:
# labs.list_refresh_requests( dataset='Supply Chain Semantic Model',workspace='ws_sci_sm_qa').dtypes

In [0]:
# import sempy.fabric as labs
# # labs.list_refresh_requests( dataset='Supply Chain Semantic Model',workspace='ws_sci_sm_qa')
# labs.get_refresh_execution_details(dataset='Supply Chain Semantic Model',workspace='ws_sci_sm_qa',refresh_request_id='bedd2011-1f67-4f9d-a926-716db71b8842')
# # labs.RefreshExecutionDetails.messages(dataset='Supply Chain Semantic Model',workspace='ws_sci_sm_qa',refresh_request_id='60213e81-97e7-4a6e-a951-2fc3d5e47aef')
# # labs.get_refresh_execution_details(dataset='Supply Chain Semantic Model',workspace='ws_sci_sm_qa')



In [0]:
# import pkg_resources

# for dist in pkg_resources.working_set:
#     print(dist.project_name)

In [0]:
import sempy.fabric as labs
import time
import pandas as pd
import pytz
from datetime import datetime,timedelta,date
from zoneinfo import ZoneInfo
import json


In [0]:
#  %%sql

# delete from lh_daily_hist.nmc_all_tab_columns
#  where lower(TABLE_NAME) = 'ms_item_ar'

In [0]:
# labs.refresh_dataset(refresh_type= 'full',objects=[{'table' : 'FN Credit Sales'},{'table':'FN Insurance'}],max_parallelism=20,apply_refresh_policy= False,retry_count=2,verbose=3,dataset = 'Finance Semantic Model' , workspace = 'ws_finance_sm_qa')
# labs.refresh_semantic_modelrefresh_type= 'full',,tables= ['Pl GM Mon Item Store'],dataset = 'Finance Semantic Model' , workspace = 'ws_finance_sm_qa')

# ,objects=[{'table' : 'Sales Day Item'}],commit_mode='partialBatch'
# ,objects=[{'table' : 'Location'},{'table' : 'SalesMonthly'}]

In [0]:
# start_time = datetime.now()
# labs.refresh_semantic_model(refresh_type= 'full',tables= ['DateTable'],dataset = 'Supply Chain Semantic Model' , workspace = 'ws_sci_sm_qa')
# labs.get_semantic_model_refresh_history(dataset = 'Supply Chain Semantic Model' , workspace = 'ws_sci_sm_qa')
# end_time = datetime.now()

In [0]:
def pre_execution_checks (dataset_id,workspace_id):

    ## 'worspace_id' check
    try :
        datasets = labs.list_datasets(workspace = workspace_id)['Dataset Name']
    except Exception as e:
        return e

    ## 'dataset_id' check
    if dataset_id in datasets.values:
        print("dataset_id Check Pass")
    else:
        return ("Dataset '" + dataset_id + "' not present in '" + workspace_id + "'.")

    ## what to do if there is an ongoing refresh?
    return 
def last_refresh_status (dataset_id,workspace_id):

    ## Returns 'False' if it is in Progress else 'True'

    pre_execution_checks(dataset_id,workspace_id)
    last_refresh = labs.list_refresh_requests(dataset = dataset_id, workspace = workspace_id).head(1)
    return ~(last_refresh['End Time'].isna())

def wait_till_refresh (dataset_id,workspace_id,request_id):

    ## Wait while the model refreshes (as per the sempy.fabric docs 'end_time' remains 'None' till the model refreshes)
    current_refresh_details =  labs.list_refresh_requests(dataset = dataset_id, workspace = workspace_id).head(1)['End Time'].values[0]

    # print(current_refresh)
    while pd.isna(current_refresh_details) :
        time.sleep(5)
        current_refresh_details =  labs.list_refresh_requests(dataset = dataset_id, workspace = workspace_id).head(1)['End Time'].values[0]
    return
def dataset_refresh(dataset_id,workspace_id):
    
    ## perform the necessary checks
    pre_execution_checks(dataset_id,workspace_id)

    ## Start the refresh

    if last_refresh_status :
        print(f"'{dataset_id}' refresh has started")
        try :
                request_id = labs.refresh_dataset(refresh_type= 'automatic',objects=[{"table": "DateTable"}],dataset = dataset_id, workspace =workspace_id,verbose =1 )
        except Exception as e:
                mssparkutils.notebook.exit(e)
        

    ## Wait for the refresh to complete
    wait_till_refresh (dataset_id,workspace_id,request_id)
    print(f"'{dataset_id}' refresh has completed ")
    
    return
def latest_refresh_stats (dataset_id,workspace_id, get_stats = True,run_refresh = False):

    if get_stats :
        df_refresh_status = {'Semantic Model' : dataset_id,
                             'Refresh Type' : 'Null',
                             'Refresh Date' : 'Null',
                            'Refresh Start Time (KSA)' :'Null',
                            'Duration': 'Null',
                            'Status': 'Null',
                            'Remarks':'Null',
                            'Error Desc': 'Null'}
        KSA = pd.Timedelta(hours= 3)
        IST = pd.Timedelta(hours = 5, minutes = 30)
        today = datetime.today().date()
        
        df_refresh_hist = labs.list_refresh_requests(dataset = dataset_id, workspace = workspace_id) 
        
        ## Get KSA timestamps for start and end timestamps

        df_refresh_hist['KSA Start Time']= pd.to_datetime(df_refresh_hist['Start Time'],utc = True) + KSA
        df_refresh_hist['KSA End Time'] =pd.to_datetime(df_refresh_hist['End Time'],utc = True) + KSA
        df_refresh_hist['Start Date'] = df_refresh_hist['KSA Start Time'].dt.date
        df_refresh_hist['Duration']= df_refresh_hist.apply(lambda x: (x['KSA End Time'] - x['KSA Start Time']).seconds, axis = 1)

        ## Get the Expected Model Refresh Duration (excluding refreshes through SSMS as they may not do the 'full' refresh)

        df_temp = df_refresh_hist[
            (df_refresh_hist['Start Date'] < today) &
            (df_refresh_hist['Refresh Type'].isin(['Scheduled', 'OnDemand', 'Via Enhanced Api']))
        ]
        df_temp = df_temp[df_temp['Status'] == 'Completed']
        avg_refresh_duration = df_temp['Duration'].mean()

        ## Filter for the current refresh ( we can also get the top 1 from the refreshes)
        # df_refresh_current = df_refresh_hist[(df_refresh_hist['KSA Start Time'] >= pd.to_datetime(start_time + KSA,utc= True) )  & ( df_refresh_hist['Refresh Type']== 'ViaEnhancedApi')]
        df_refresh_current = df_refresh_hist.head(1)
        
        
        
        df_refresh_status['Refresh Type'] = df_refresh_current['Refresh Type'][0]
        df_refresh_status['Refresh Date'] = df_refresh_current ['KSA Start Time'].dt.strftime('%Y-%m-%d').values[0]
        df_refresh_status['Refresh Start Time (KSA)'] = df_refresh_current['KSA Start Time'].dt.strftime('%I:%M:%S %p').values[0]

        ## If the pipeline is run for only stats then don't stop for the refresh to complete, instead go ahead with run in progress as a remark
        if pd.isna(df_refresh_current['End Time'].values[0]):
            df_refresh_status['Status'] = 'In Progress'
            df_refresh_status['Remarks'] = '🔄 Refresh is in Progess, Hence skipped the few metrics.'
            return df_refresh_status
            
        df_refresh_status['Status'] = df_refresh_current['Status'][0]
      
        current_refresh_status = df_refresh_current['Duration'].mean()
        ## Model Duration Sanity Check
        if 90 <= (current_refresh_status*100.00/avg_refresh_duration) <= 110:
                df_refresh_status['Remarks'] = '✅ Refresh Completed in Time'
        elif (current_refresh_status*100.00/avg_refresh_duration) < 90 :
                df_refresh_status['Remarks'] = '✅ Refresh Completed Quickly'
        else  :
                df_refresh_status['Remarks'] = '🟡 Refresh Took Longer Than Expected'

        if current_refresh_status >= 3600 :
             current_refresh_status = f"{int(current_refresh_status//3600)}h {int((current_refresh_status%3600)//60)} Min {int(current_refresh_status % 60)} Sec" 
        else :
            current_refresh_status = f"{int(current_refresh_status//60)} Min {int(current_refresh_status % 60)} Sec" 
        ## Duration 
        df_refresh_status['Duration'] = current_refresh_status 

        # df_refresh_status['Model Size'] = labs.get_semantic_model_size(dataset=dataset, workspace=workspace) / (1024*1024*1024)


        ## Check the Model Referesh if it is failed
        df_refresh_current_f = df_refresh_current[df_refresh_current['Status'] == 'Failed']
        if len(df_refresh_current_f) > 0:
            df_refresh_status['Remarks'] = '🔴 Refresh Failed!!'
            df_refresh_status['Error Desc'] = df_refresh_current_f['Service Exception Json'][0]

        ## Check the Model Referesh if it is Canceled
        df_refresh_current_c = df_refresh_current[df_refresh_current['Status'] == 'Cancelled']
        if len(df_refresh_current_c) > 0:         
            df_refresh_status['Remarks'] = '⛔️ Refresh Cancelled by User!!'
            df_refresh_status['Error Desc'] = df_refresh_current_f['Service Exception Json'][0]
        return df_refresh_status
    return "Stats Not Requested"


In [0]:
start_time = datetime.now()

if run_refresh == True:
    dataset_refresh(dataset,workspace)
status = latest_refresh_stats (dataset,workspace, get_stats)
if type(status) == dict:
    status = f"<tr><th>{workspace}</th><th> {status['Semantic Model']}</th><th>{status['Refresh Type'] }</th><th>{status['Refresh Date']} </th><th>{status['Refresh Start Time (KSA)']} </th><th>{status['Duration']} </th><th> {status['Status']}</th><th>{status['Remarks']} </th><th>{status['Error Desc']} </th></tr>"
            

mssparkutils.notebook.exit(status)


In [0]:
# dataset = 'Supply Chain Semantic Model'
# workspace = 'ws_sci_sm_prd'
# latest_refresh_stats(dataset,workspace,True)
# # # .

# # latest_refresh_stats (dataset,workspace, True)



In [0]:
# df_refresh_status = {'Semantic Model' : dataset,
#                      'Refresh Start (KSA)' :None,
#                      'Duration': None,
#                      'Refresh Type' : None,
#                      'Status': None,
#                      'Remarks':None,
#                      'Error Code': None,
#                      'Error Desc': None}
# KSA = pd.Timedelta(hours= 2,minutes = 55)
# IST = pd.Timedelta(hours = 5, minutes = 30)
# today = datetime.today().date()
# df_refresh_hist = labs.list_refresh_requests(dataset = dataset, workspace = workspace)

# ## Get KSA timestamps for start and end timestamps

# df_refresh_hist['KSA Start Time']= pd.to_datetime(df_refresh_hist['Start Time'],utc = True) + KSA
# df_refresh_hist['KSA End Time'] =pd.to_datetime(df_refresh_hist['End Time'],utc = True) + KSA
# df_refresh_hist['Start Date'] = df_refresh_hist['KSA Start Time'].dt.date
# df_refresh_hist['Duration']= df_refresh_hist.apply(lambda x: (x['KSA End Time'] - x['KSA Start Time']).seconds, axis = 1)

# ## Get the Expected Model Refresh Duration (excluding refreshes through SSMS as they may not do the 'full' refresh)

# df_temp = df_refresh_hist[
#     (df_refresh_hist['Start Date'] < today) &
#     (df_refresh_hist['Refresh Type'].isin(['Scheduled', 'OnDemand', 'Via Enhanced Api']))
# ]
# df_temp = df_temp[df_temp['Status'] == 'Completed']
# avg_refresh_duration = df_temp['Duration'].mean()

# ## Filter for the current refresh
# df_refresh_current = df_refresh_hist[(df_refresh_hist['KSA Start Time'] >= pd.to_datetime(start_time + KSA,utc= True) ) & ( df_refresh_hist['KSA End Time'] <= pd.to_datetime(end_time+KSA,utc = True) ) & ( df_refresh_hist['Refresh Type']== 'ViaEnhancedApi')]
# current_refresh_status = df_refresh_current['Duration'].mean()



# ## Model Duration Sanity Check
# if len(df_refresh_current) > 0 :
#     if 90 <= (current_refresh_status*100.00/avg_refresh_duration) <= 110:
#         df_refresh_status['Remarks'] = 'Refresh Completed in Time'
#     elif (current_refresh_status*100.00/avg_refresh_duration) < 90 :
#         df_refresh_status['Remarks'] = 'Refresh Completed Quickly'
#     else  :
#         df_refresh_status['Remarks'] = 'Refresh Took Longer Than Expected'

#     df_refresh_status['Duration'] = current_refresh_status
#     df_refresh_status['Refresh Type'] = 'Via Enhanced Api'

# df_refresh_status['Remarks'] = 'Refresh '
# # df_refresh_status['Model Size'] = labs.get_semantic_model_size(dataset=dataset, workspace=workspace) / (1024*1024*1024)

# ## Check the Model Referesh if it is Completed
# df_refresh_current_s = df_refresh_current[df_refresh_current['Status'] == 'Completed']
# if len(df_refresh_current_s)> 0 :
#     df_refresh_status['Refresh Start (KSA)'] =   df_refresh_current_s['KSA Start Time'][0]
#     df_refresh_status['Status']= 'Completed'
    

# ## Check the Model Referesh if it is failed
# df_refresh_current_f = df_refresh_current[df_refresh_current['Status'] == 'Failed']
# if len(df_refresh_current_f) > 0:
#     df_refresh_status['Refresh Start (KSA)'] =   df_refresh_current_f['KSA Start Time'][0]
#     df_refresh_status['Duration'] =  df_refresh_current_f['Duration'][0]
#     df_refresh_status['Refresh Type'] = 'Via Enhanced Api'
#     df_refresh_status['Status']= 'Failed'
#     df_refresh_status['Remarks'] = 'Refresh Failed!!'
#     df_refresh_status['Error Code'] = df_refresh_current_f['Error Code'][0],
#     df_refresh_status['Error Desc'] = df_refresh_current_f['Error Description'][0]

# ## Check the Model Referesh if it is Canceled
# df_refresh_current_c = df_refresh_current[df_refresh_current['Status'] == 'Cancelled']
# if len(df_refresh_current_c) > 0:
#     df_refresh_status['Refresh Start (KSA)'] =   df_refresh_current_c['KSA Start Time'][0]
#     df_refresh_status['Duration'] =  df_refresh_current_f['Duration'][0]
#     df_refresh_status['Refresh Type'] = 'Via Enhanced Api'
#     c
#     df_refresh_status['Remarks'] = 'Refresh Unsuccessful!!'
#     df_refresh_status['Error Code'] = df_refresh_current_f['Error Code'][0],
#     df_refresh_status['Error Desc'] = df_refresh_current_f['Error Description'][0]

# mssparkutils.notebook.exit(df_refresh_status)

In [0]:
# model_dict = {
            #   'FTG Semantic Model'        : { 'dataset' : 'FTG Semantic Model'          , 'workspace' : 'ws_retail_sm_qa' },
#             'Global Semantic Model'       : { 'dataset' : 'Global Semantic Model'       , 'workspace' : 'ws_retail_sm_qa' },
#             'HR Semantic Model'           : { 'dataset' : 'HR Semantic Model'           , 'workspace' : 'ws_hr_sm_qa' },
#             'Supply Chain Semantic Model' : { 'dataset' : 'Supply Chain Semantic Model' , 'workspace' : 'ws_sci_sm_qa' },
#             'Initiative Semantic Model'   : { 'dataset' : 'Initiative Semantic Model'   , 'workspace' : 'ws_retail_sm_qa' },
#             'Quality Semantic Model'      : { 'dataset' : 'Quality Semantic Model'      , 'workspace' : 'ws_retail_sm_qa' },
#             'Sales Semantic Model'        : { 'dataset' : 'Sales Semantic Model'        , 'workspace' : 'ws_retail_sm_qa' },
#             'Finance Semantic Model'      : { 'dataset' : 'Finance Semantic Model'      , 'workspace' : 'ws_finance_sm_qa' },
#             'Inventory Semantic Model'    : { 'dataset' : 'Inventory_Semantic_Model'    , 'workspace' : 'ws_sci_sm_qa' },
#             'Payment Semantic Model'      : { 'dataset' : 'Payment Semantic Model'      , 'workspace' : 'ws_retail_sm_qa' },
#             'Clinics Semantic Model'      : { 'dataset' : 'Clinics Semantic Model'      , 'workspace' : 'ws_clinics_sm_qa' }
#             }


In [0]:
# print("---> Pinging " + model_dict[k]['dataset'])
#     df_time_zone = labs.get_semantic_model_refresh_schedule(dataset=model_dict[k]['dataset'], workspace=model_dict[k]['workspace'])
#     df_time_zone = df_time_zone[df_time_zone['Refresh Type'] == 'Scheduled']
#     df_time_zone['KSA Start Time']= pd.to_datetime(df_time_zone['Start Time'],utc = True) + KSA
#     df_time_zone['KSA End Time'] =pd.to_datetime(df_time_zone['End Time'],utc = True) + KSA
#     df_time_zone['IST Start Time']= pd.to_datetime(df_time_zone['Start Time'],utc = True) + IST
#     df_time_zone['IST End Time'] =pd.to_datetime(df_time_zone['End Time'],utc = True) + IST
#     df_time_zone['Duration']=df.apply(lambda x: (x['IST End Time'] - x['IST Start Time']).seconds, axis = 1)
#     model_sizes = pd.concat([
#     model_sizes,
#     pd.DataFrame({
#         'Semantic Model': [model_dict[k]['dataset']],

In [0]:
# labs.refresh_semantic_model(dataset=dataset, workspace=workspace) 

In [0]:
# mssparkutils.notebook.exit((dataset,workspace))